In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cluster import KMeans

import plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt 

In [ ]:
zoo = pd.read_csv(r'/kaggle/input/zoo-animal-classification/zoo.csv')

In [ ]:
zoo.describe()

## Determining the number of clusters (I chose 3)

In [ ]:
vals = zoo.drop(['animal_name'],axis=1).values
inertia = []
for n in range(1 , 11):
    clust = KMeans(n_clusters = n)
    clust.fit(vals)
    inertia.append(clust.inertia_)

In [ ]:
plt.figure(1 , figsize = (15 ,6))
plt.plot(np.arange(1 , 11) , inertia , 'o')
plt.plot(np.arange(1 , 11) , inertia , '-' , alpha = 0.5)
plt.xlabel('Number of Clusters') , plt.ylabel('Inertia')
plt.show()

## Visualizing Clusters

In [ ]:
zoo_cluster = KMeans(n_clusters = 3)# ,init='k-means++', n_init = 10 ,max_iter=300, tol=0.0001,  random_state= 111  , algorithm='elkan')
zoo_cluster.fit(vals)
labs = zoo_cluster.labels_
centroids = zoo_cluster.cluster_centers_

In [ ]:
zoo['label'] =  labs
trace = go.Scatter3d(
    x= zoo['predator'],
    y= zoo['class_type'],
    z= zoo['legs'],
    mode='markers',
     marker=dict(
        color = zoo['label'], 
        size= 5,
        line=dict(
            color= zoo['label'],
            width= 12
        ),
        opacity=0.8
     )
)
data = [trace]
layout = go.Layout(
    title= 'Clusters',
    scene = dict(
            xaxis = dict(title  = 'predator'),
            yaxis = dict(title  = 'class'),
            zaxis = dict(title  = 'legs')
        )
)
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)

# Finding Weird Animals

In [ ]:
df = zoo.drop(['animal_name'], axis=1)

zoo_cluster2 = KMeans(n_clusters=5)
zoo_cluster2.fit(df.values)
centroids2 = zoo_cluster2.cluster_centers_
labs = zoo_cluster2.labels_

df['cluster'] = labs
zoo['cluster'] = labs

In [ ]:
for i, cent in enumerate(centroids2):
    dist = np.sqrt(np.square(centroids2[i] - df[df['cluster'] == i].drop(['cluster'], axis=1).values).sum(axis=1))
    ind = np.argmax(dist)
    print(zoo[zoo['cluster'] == i].reset_index().iloc[ind].animal_name, ' has a distance: ', dist.max())